### Retrieving structural data from the PDB using GraphQL API

Prior to running this script, an advanced search was performed on the RCSB Protein Data Bank (PDB), filtering for structures containing the Kunitz-type protease inhibitor domain (Pfam: PF00014), with resolution ≤ 3.5 Å and sequence length between 45 and 80 amino acids. From the *Custom Report* interface, a GraphQL query was generated based on this filtered set, yielding a list of 159 PDB IDs.

This Python script takes that list of PDB IDs and submits it in batches of 100 to the RCSB GraphQL API. For each structure, it retrieves:
- Entry-level resolution data  
- Polymer chain identifiers  
- Canonical amino acid sequences  
- Pfam domain annotations  

The output is saved to a JSON file (`all_kunitz_structures.json`), which consolidates all retrieved metadata and sequences for use in downstream filtering, alignment, and HMM construction.


In [ ]:
import requests
import json
import time

# === PARAMETRI ===
pdb_ids = [
    "1AAL", "1AAP", "1B0C", "1BHC", "1BPI", "1BPT", "1BRC", "1BTH", "1BTI", "1BUN", "1BZ5", "1BZX", "1CA0", "1CBW", "1D0D", "1DTX",
    "1EAW", "1EJM", "1F5R", "1F7Z", "1FAK", "1FAN", "1FY8", "1G6X", "1K6U", "1KNT", "1KTH", "1MTN", "1NAG", "1P2I", "1P2J", "1P2K",
    "1P2M", "1P2N", "1P2O", "1P2Q", "1QLQ", "1T7C", "1T8L", "1T8M", "1T8N", "1T8O", "1TAW", "1TFX", "1TPA", "1Y62", "1YC0", "1YKT",
    "1YLC", "1YLD", "1ZJD", "1ZR0", "2FI3", "2FI4", "2FI5", "2FTL", "2FTM", "2HEX", "2IJO", "2KAI", "2KNT", "2ODY", "2PTC", "2R9P",
    "2RA3", "2TGP", "2TPI", "2ZJX", "2ZVX", "3BTD", "3BTE", "3BTF", "3BTG", "3BTH", "3BTK", "3BTM", "3BTQ", "3BTT", "3BTW", "3BYB",
    "3D65", "3FP6", "3FP7", "3FP8", "3GYM", "3L33", "3LDI", "3LDJ", "3LDM", "3M7Q", "3OFW", "3OTJ", "3P92", "3P95", "3T62", "3TGI",
    "3TGJ", "3TGK", "3TPI", "3U1J", "3UIR", "3UOU", "3WNY", "4BQD", "4DG4", "4DTG", "4ISL", "4ISN", "4ISO", "4NTW", "4NTX", "4NTY",
    "4PTI", "4TPI", "4U30", "4U32", "4WWY", "4WXV", "4Y0Y", "4Y0Z", "4Y10", "4Y11", "5JB4", "5JB5", "5JB6", "5JB7", "5JBT", "5M4V",
    "5NX1", "5NX3", "5PTI", "5XX2", "5XX3", "5XX4", "5XX5", "5XX6", "5XX7", "5XX8", "5YV7", "5YVU", "5YW1", "5ZJ3", "6BX8", "6F1F",
    "6HAR", "6KZF", "6PTI", "6Q61", "6Q6C", "6YHY", "7PH1", "7PTI", "7QIQ", "7QIR", "7QIS", "7QIT", "8PTI", "8VC3", "9PTI"
]

# === Funzione per dividere la lista in blocchi da 100 ===
def chunk_list(lst, size):
    for i in range(0, len(lst), size):
        yield lst[i:i + size]

# === Endpoint GraphQL ===
url = "https://data.rcsb.org/graphql"

# === Struttura della query GraphQL ===
def make_query(entry_ids):
    ids_formatted = ",".join(f'"{pid}"' for pid in entry_ids)
    return {
        "query": f"""
        {{
          entries(entry_ids: [{ids_formatted}]) {{
            rcsb_id
            rcsb_entry_info {{
              resolution_combined
            }}
            polymer_entities {{
              entity_poly {{
                pdbx_seq_one_letter_code_can
                rcsb_sample_sequence_length
              }}
              rcsb_polymer_entity_annotation {{
                annotation_id
                type
              }}
              polymer_entity_instances {{
                rcsb_polymer_entity_instance_container_identifiers {{
                  auth_asym_id
                }}
              }}
            }}
          }}
        }}
        """
    }

# === Inizializza il contenitore per tutte le risposte ===
all_entries = []

# === Esegui la query per ogni batch ===
for idx, chunk in enumerate(chunk_list(pdb_ids, 100), start=1):
    print(f" Downloading batch {idx} with {len(chunk)} IDs...")
    response = requests.post(url, json=make_query(chunk))
    if response.status_code == 200:
        result = response.json()
        entries = result.get("data", {}).get("entries", [])
        all_entries.extend(entries)
        time.sleep(1)  # un attimo di pausa tra richieste
    else:
        print(f" Errore batch {idx}:", response.status_code, response.text)

# === Salva il file finale ===
with open("all_kunitz_structures.json", "w") as f:
    json.dump({"entries": all_entries}, f, indent=2)

print(f"\n Salvato: all_kunitz_structures.json con {len(all_entries)} strutture.")



### Filtering and exporting Kunitz sequences in FASTA format

To prepare the retrieved dataset for downstream analysis, we developed a second script (`filter_kunitz_to_fasta.py`) that parses the `all_kunitz_structures.json` file and extracts only high-quality, non-redundant sequences matching the desired structural and domain criteria.

The script applies the following filters:
1. The sequence must contain the Pfam domain **PF00014**
2. The associated structure must have an experimental resolution of **≤ 3.5 Å**
3. The amino acid sequence must be **between 45 and 80 residues** in length
4. The sequence must be **unique** (no duplicates allowed)

For each entry passing all criteria, the script extracts the canonical amino acid sequence and writes it to a FASTA-formatted file (`filtered_kunitz_sequences.fasta`). Each header line includes metadata such as the PDB ID, chain identifier(s), resolution, and sequence length.

This filtered and deduplicated FASTA file serves as the input for downstream clustering with **CD-HIT**, alignment, and HMM construction.

We retrieved 85 sequences.

In [ ]:
import json

# === PARAMETRI ===
input_file = "all_kunitz_structures.json"
output_file = "filtered_kunitz_sequences.fasta"
min_length = 45
max_length = 80
max_resolution = 3.5
required_pfam = "PF00014"

# === CARICA IL FILE JSON ===
with open(input_file, "r") as f:
    data = json.load(f)

entries = data.get("entries", [])
seen_sequences = set()
count = 0

# === CREA FILE FASTA ===
with open(output_file, "w") as fasta:
    for entry in entries:
        pdb_id = entry.get("rcsb_id", "N/A")
        resolutions = entry.get("rcsb_entry_info", {}).get("resolution_combined", [])
        resolution = resolutions[0] if resolutions else None
        if resolution is None or resolution > max_resolution:
            continue

        for entity in entry.get("polymer_entities", []):
            sequence = entity.get("entity_poly", {}).get("pdbx_seq_one_letter_code_can", "").strip()
            length = entity.get("entity_poly", {}).get("rcsb_sample_sequence_length", 0)

            # Controllo lunghezza e presenza sequenza
            if not sequence or sequence in seen_sequences:
                continue
            if not (min_length <= length <= max_length):
                continue

            # Controllo dominio PF00014
            annotations = entity.get("rcsb_polymer_entity_annotation", [])
            pfam_ids = [ann.get("annotation_id") for ann in annotations if ann.get("type") == "Pfam"]
            if required_pfam not in pfam_ids:
                continue

            seen_sequences.add(sequence)
            count += 1

            # Catena
            chains = [
                inst.get("rcsb_polymer_entity_instance_container_identifiers", {}).get("auth_asym_id", "N/A")
                for inst in entity.get("polymer_entity_instances", [])
            ]
            chain_info = ",".join(chains) if chains else "N/A"

            # Scrivi FASTA
            header = f">seq{count} | PDB:{pdb_id} | Chain:{chain_info} | Res:{resolution} | Len:{length}"
            fasta.write(f"{header}\n{sequence}\n")

print(f" {count} sequences written to {output_file}")



### CD-HIT clustering, structural alignment and HMM construction

The 85 non-identical Kunitz sequences were clustered using CD-HIT at 70% identity to remove redundancy.

The resulting 19 representative sequences (from pdb_kunitz_nr.fasta) were extracted from headers to create the PDBeFold input.
PDBeFold multiple alignment revealed poor alignment for 1bun:B, 4ntw:B, 4u30:X (RMSD > 1.0) and 4bqd:A (Q-score < 0.7), which were excluded.
The remaining 15 structures were exported as a structural MSA (kunitz_15_seq_structural_MSA.fasta).
Due to an abnormally long sequence (1dtx:A), we removed it to avoid build errors.

The final structural MSA was used to build the HMM with HMMER

The final model is based on 14 structurally aligned sequences and covers 58 match states, consistent with the typical size of the Kunitz domain.

```bash
cd-hit -i filtered_kunitz_sequences.fasta -o pdb_kunitz_nr.fasta -c 0.7 -n 5 \```


awk 'BEGIN{RS=">"; ORS=""} NR==1{next} $0 !~ /^PDB:1dtx:A/ {print ">" $0}' kunitz_15_seq_structural_MSA.fasta > kunitz_clean_14_MSA.fasta


grep -c "^>" kunitz_clean_14_MSA.fasta


hmmbuild kunitz_structural.hmm kunitz_clean_14_MSA.fasta

In [ ]:
"""
Questo script carica i risultati di un confronto strutturale (output di PDBeFold)
contenuti in un file CSV e identifica eventuali outlier in base a soglie definite
per RMSD (Root Mean Square Deviation) e Q-score.

In particolare:
- I punti con RMSD > 1.0 o Q-score < 0.7 sono considerati outlier.
- Viene generato un grafico scatterplot con RMSD sull'asse x e Q-score sull'asse y.
- Gli outlier sono evidenziati in rosso, gli altri in blu.
- Due linee tratteggiate mostrano le soglie utilizzate per l'identificazione.

Il plot risultante aiuta a visualizzare la distribuzione delle strutture
e a individuare rapidamente eventuali allineamenti strutturali anomali.
"""



import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Soglie per individuare outlier
rmsd_cutoff = 1.0
qscore_cutoff = 0.7

# Carica il file CSV corretto
df = pd.read_csv("pdbe_fold_results_clean.csv")

# Identifica gli outlier
df["is_outlier"] = (df["RMSD"] > rmsd_cutoff) | (df["Qscore"] < qscore_cutoff)

# Plot
plt.figure(figsize=(9, 6))
sns.scatterplot(data=df, x="RMSD", y="Qscore", hue="is_outlier", palette={True: "red", False: "blue"}, s=100)

# Linee di soglia
plt.axvline(x=rmsd_cutoff, color="gray", linestyle="--", label=f"RMSD = {rmsd_cutoff}")
plt.axhline(y=qscore_cutoff, color="gray", linestyle="--", label=f"Q-score = {qscore_cutoff}")

# Etichette
plt.title("Structural Alignment – RMSD vs Q-score")
plt.xlabel("RMSD")
plt.ylabel("Q-score")
plt.grid(True, linestyle="--", linewidth=0.5)
plt.legend(title="Outlier")
plt.tight_layout()
plt.show()




---


```markdown
### Validation Phase: Filtering, Test Set Construction and Performance Evaluation

In this phase, we constructed non-redundant positive and negative test sets for evaluating the HMM’s predictive performance using cross-validation on SwissProt data. Redundant sequences were excluded using BLAST, and balanced test sets were created. The model was evaluated with `hmmsearch` and metrics computed with `score.py`.

**Steps overview:**

1) Create BLAST database from SwissProt  
2) Align training sequences against the database  
3) Filter redundant hits (≥95% identity, ≥50 aa)  
4) Remove redundant sequences from the positive set  
5) Extract all SwissProt IDs  
6) Extract final Kunitz-positive IDs  
7) Generate the list of negative IDs (SwissProt – Kunitz)  
8) Shuffle positive and negative ID lists  
9) Count total entries for each  
10) Split positives into two halves (pos_1, pos_2)  
11) Split negatives into two halves (neg_1, neg_2)  
12) Extract FASTA sequences using ID lists  
13) Run `hmmsearch` on all subsets  
14) Generate .class files from output  
15) Add missing negatives to .class with default values  
16) Merge and sort class files  
17) Combine positive and negative .class files into set_1 and set_2  
18) Run scoring at threshold 1e-5  
19) Scan thresholds from 1e-1 to 1e-30 and log results



```bash
makeblastdb -in all_kunitz_sprot.fasta -dbtype prot -out all_kunitz_sprot ```

blastp -query kunitz_clean_14_MSA.fasta -db all_kunitz_sprot -out pdb_kunitz_nr_14.blast -outfmt 6

awk '$3 >= 95 && $4 >= 50 {print $2}' pdb_kunitz_nr_14.blast | sort -u > ids_to_remove.txt

python filter_fasta_remove_ids.py all_kunitz_sprot.fasta ids_to_remove.ids filtered_kunitz_sprot.fasta

grep ">" uniprot_sprot-2.fasta | cut -d "|" -f2 > all_sprot.ids
grep ">" filtered_kunitz_sprot.fasta | cut -d "|" -f2 > all_kunitz.id
comm -23 <(sort all_sprot.ids) <(sort all_kunitz.id) > negs.id
sort -R all_kunitz.id > random_all_kunitz.ids
sort -R negs.id > random_negs.id
wc -l random_all_kunitz.ids
wc -l random_negs.id
head -n 194 random_all_kunitz.ids > pos_1.ids
tail -n +195 random_all_kunitz.ids > pos_2.ids
head -n 286421 random_negs.id > neg_1.ids
tail -n +286422 random_negs.id > neg_2.ids

python3 extract_fasta_by_ids.py pos_1.ids uniprot_sprot-2.fasta > pos_1.fasta
python3 extract_fasta_by_ids.py pos_2.ids uniprot_sprot-2.fasta > pos_2.fasta
python3 extract_fasta_by_ids.py neg_1.ids uniprot_sprot-2.fasta > neg_1.fasta
python3 extract_fasta_by_ids.py neg_2.ids uniprot_sprot-2.fasta > neg_2.fasta

hmmsearch -Z 1000 --max --tblout pos_1.out kunitz_structural.hmm pos_1.fasta
hmmsearch -Z 1000 --max --tblout pos_2.out kunitz_structural.hmm pos_2.fasta
hmmsearch -Z 1000 --max --tblout neg_1.out kunitz_structural.hmm neg_1.fasta
hmmsearch -Z 1000 --max --tblout neg_2.out kunitz_structural.hmm neg_2.fasta

grep -v "^#" pos_1.out | awk '{split($1,a,"|"); print a[2]"\t1\t"$5"\t"$8}' > pos_1.class
grep -v "^#" pos_2.out | awk '{split($1,a,"|"); print a[2]"\t1\t"$5"\t"$8}' > pos_2.class
grep -v "^#" neg_1.out | awk '{split($1,a,"|"); print a[2]"\t0\t"$5"\t"$8}' > neg_1.class
grep -v "^#" neg_2.out | awk '{split($1,a,"|"); print a[2]"\t0\t"$5"\t"$8}' > neg_2.class

comm -23 <(sort neg_1.ids) <(cut -f1 neg_1.class | sort) > neg_1_missing.ids
wc -l neg_1_missing.ids
awk '{print $1"\t0\t10.0\t10.0"}' neg_1_missing.ids > neg_1_missing.class
cat neg_1.class neg_1_missing.class > neg_1_complete.class
sort neg_1_complete.class > neg_1.class
comm -23 <(sort neg_2.ids) <(cut -f1 neg_2.class | sort) > neg_2_missing.ids
wc -l neg_2_missing.ids
awk '{print $1"\t0\t10.0\t10.0"}' neg_2_missing.ids > neg_2_missing.class
cat neg_2.class neg_2_missing.class > neg_2_complete.class
sort neg_2_complete.class > neg_2.class
cat pos_1.class neg_1.class > set_1.class
cat pos_2.class neg_2.class > set_2.class

python3 score.py set_1.class 1e-5
python3 score.py set_2.class 1e-5
for i in $(seq 1 30); do python3 score.py set_1.class 1e-$i >> results_set1.txt; done
for i in $(seq 1 30); do python3 score.py set_2.class 1e-$i >> results_set2.txt; done


In the following 3 cells,there are the three scripts mentioned in the previous markdown: filter_fasta_remove_ids.py, extract_fasta_by_ids.py, score.py

In [ ]:
# filter_fasta_remove_ids.py

# This script removes sequences from a FASTA file based on a list of sequence IDs.
# It is used to filter out redundant sequences (e.g. highly similar to training data) 
# from a positive test set before evaluating the HMM.


from Bio import SeqIO
import sys

fasta_file = sys.argv[1]         # es. all_kunitz_sprot.fasta
ids_file = sys.argv[2]           # es. ids_to_remove.ids
output_file = sys.argv[3]        # es. filtered_kunitz_sprot.fasta

# Carica gli ID da rimuovere
with open(ids_file) as f:
    ids_to_remove = set(line.strip() for line in f if line.strip())

# Filtra le sequenze
with open(fasta_file) as fin, open(output_file, "w") as fout:
    for record in SeqIO.parse(fin, "fasta"):
        if record.id not in ids_to_remove:
            SeqIO.write(record, fout, "fasta")


In [ ]:
#!/usr/bin/env python3

# This script extracts sequences from a FASTA file based on a list of target IDs.
# It is used to build positive and negative test sets by selecting specific sequences
# from the full SwissProt database using predefined ID lists.


import sys

def get_ids(idlist_file):
    with open(idlist_file) as f:
        return set(f.read().strip().split('\n'))

def get_seq(pidlist, seqfile):
    with open(seqfile) as f:
        print_flag = False
        for line in f:
            if line.startswith('>'):
                parts = line.strip().split('|')
                pid = parts[1] if len(parts) > 1 else line.strip()[1:]
                print_flag = pid in pidlist
            if print_flag:
                print(line.strip())

if __name__ == '__main__':
    idlist_file = sys.argv[1]
    seqfile = sys.argv[2]
    pidlist = get_ids(idlist_file)
    get_seq(pidlist, seqfile)



In [ ]:
#!/usr/bin/env python3

# This script evaluates HMM classification performance by parsing .class files.
# It calculates metrics such as sensitivity, specificity, precision, accuracy, 
# F1-score, and Matthews Correlation Coefficient (MCC) at a specified E-value threshold.

import sys

threshold = float(sys.argv[2])

TP = 0
FP = 0
TN = 0
FN = 0

with open(sys.argv[1]) as f:
    for line in f:
        line = line.strip()
        if line == "":
            continue
        fields = line.split()
        ID = fields[0]
        real = int(fields[1])
        score = float(fields[2])

        pred = 1 if score < threshold else 0

        if real == 1 and pred == 1:
            TP += 1
        elif real == 1 and pred == 0:
            FN += 1
        elif real == 0 and pred == 1:
            FP += 1
        elif real == 0 and pred == 0:
            TN += 1

# Calculate metrics
total = TP + FP + TN + FN
acc = (TP + TN) / total if total > 0 else 0
mcc_num = (TP * TN) - (FP * FN)
mcc_den = ((TP + FP)*(TP + FN)*(TN + FP)*(TN + FN))**0.5
mcc = mcc_num / mcc_den if mcc_den != 0 else 0
tpr = TP / (TP + FN) if (TP + FN) > 0 else 0
fpr = FP / (FP + TN) if (FP + TN) > 0 else 0

# Print results
print(f"Threshold: {threshold}")
print("Confusion Matrix:")
print(f"    True Positives (TP): {TP}")
print(f"    False Positives (FP): {FP}")
print(f"    True Negatives (TN): {TN}")
print(f"    False Negatives (FN): {FN}\n")

print(f"Accuracy: {acc:.6f}")
print(f"Matthews Correlation Coefficient (MCC): {mcc:.6f}")
print(f"True Positive Rate (Sensitivity): {tpr:.6f}")
print(f"False Positive Rate: {fpr:.6f}")



### Two-Fold Cross Validation and ROC Analysis – Commands

```bash
# Run scoring on both sets at fixed threshold
python3 score.py set_1.class 1e-5
python3 score.py set_2.class 1e-5

# Scan multiple thresholds to find optimal MCC
for i in $(seq 1 30); do python3 score.py set_1.class 1e-$i >> results_set1.txt; done
for i in $(seq 1 30); do python3 score.py set_2.class 1e-$i >> results_set2.txt; done

# Run custom script to generate ROC curve and calculate AUC
python3 roc_analysis.py set_1.class
python3 roc_analysis.py set_2.class


In [ ]:
# This script plots the Matthews Correlation Coefficient (MCC) as a function of the E-value threshold
# for two validation sets. It visualizes how model performance varies with threshold selection and 
# highlights the chosen optimal threshold (e.g., 1e-5) using a vertical dashed red line.
# Useful for comparing MCC trends across thresholds and justifying the selected cutoff in cross-validation.

import numpy as np

# Threshold ottimale scelto nel report (es. 1e-5)
best_threshold = 0,1

# Plot con linea rossa verticale per il best threshold
plt.figure(figsize=(10, 6))
plt.plot(thresholds1, mcc1, marker='o', label='Validation Set 1')
plt.plot(thresholds2, mcc2, marker='s', label='Validation Set 2')
plt.axvline(x=best_threshold, color='red', linestyle='--', label=f'Chosen threshold = {best_threshold:0.1}')
plt.xscale('log')
plt.xlabel('Threshold (log scale)')
plt.ylabel('Matthews Correlation Coefficient (MCC)')
plt.title('MCC vs Threshold with Chosen Cutoff')
plt.grid(True, which='both', linestyle='--', linewidth=0.5)
plt.legend()
plt.tight_layout()
plt.show()



In [ ]:
# This script generates side-by-side confusion matrix heatmaps for the two validation sets (Set 1 and Set 2)
# evaluated at the selected optimal threshold (0.01). Each matrix summarizes the number of true positives (TP),
# false negatives (FN), false positives (FP), and true negatives (TN), providing an intuitive overview of 
# the model’s classification performance on each subset. The use of color-coded heatmaps helps to highlight 
# differences in prediction accuracy across sets. The output is also saved as a high-resolution PNG image.
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Confusion matrix values
# Format: [[TP, FN], [FP, TN]]
cm_set1 = np.array([[194, 0],
                    [0, 286421]])

cm_set2 = np.array([[193, 1],
                    [0, 286421]])

# Etichette degli assi
labels = ["Positive", "Negative"]

# Crea il grafico con 2 subplot
fig, axs = plt.subplots(1, 2, figsize=(12, 5))

# Confusion matrix - Set 1
sns.heatmap(cm_set1, annot=True, fmt="d", cmap="Blues", 
            xticklabels=labels, yticklabels=labels, ax=axs[0])
axs[0].set_title("Confusion Matrix - Set 1 (Threshold = 0.01)")
axs[0].set_xlabel("Predicted Label")
axs[0].set_ylabel("True Label")

# Confusion matrix - Set 2
sns.heatmap(cm_set2, annot=True, fmt="d", cmap="Greens", 
            xticklabels=labels, yticklabels=labels, ax=axs[1])
axs[1].set_title("Confusion Matrix - Set 2 (Threshold = 0.01)")
axs[1].set_xlabel("Predicted Label")
axs[1].set_ylabel("True Label")

# Layout ordinato e salvataggio opzionale
plt.tight_layout()
plt.savefig("confusion_matrices_set1_set2.png", dpi=300)
plt.show()



In [ ]:
# This script generates a ROC (Receiver Operating Characteristic) curve from a .class file containing 
# classification results from HMMER (sequence ID, true class label, and E-values). 
# It transforms E-values into confidence scores using the negative logarithm, computes the true positive 
# rate (TPR) and false positive rate (FPR) at varying thresholds, and calculates the area under the curve (AUC).
# The resulting ROC plot is saved as a PNG image named after the input file.
# Usage: python3 plot_roc_curve.py set_1.class


import pandas as pd
import numpy as np
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
import sys
import os

# --- Check argomenti ---
if len(sys.argv) != 2:
    print("Usage: python3 plot_roc_curve.py <set_file.class>")
    sys.exit(1)

input_file = sys.argv[1]

# Estrai nome base (es: set_1) per salvare immagine
base_name = os.path.splitext(os.path.basename(input_file))[0]
output_image = f"roc_curve_{base_name}.png"

# --- Carica i dati ---
df = pd.read_csv(input_file, sep="\t", header=None, names=["ID", "Class", "Evalue1", "Evalue2"])

# Trasforma E-value in score logaritmico (più basso = predizione migliore)
df["score"] = -np.log10(df["Evalue1"].astype(float))

# Calcola ROC
fpr, tpr, thresholds = roc_curve(df["Class"], df["score"])
roc_auc = auc(fpr, tpr)

# --- Plot ROC ---
plt.figure(figsize=(7, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.3f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Random')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title(f'ROC Curve for {base_name}')
plt.legend(loc="lower right")
plt.grid(True)

# Salva immagine
plt.savefig(output_image, dpi=300)
print(f"ROC curve saved to {output_image}")
plt.show()

